In [1]:
import numpy as np
from Robot import UR5Arm
from fwdkin import fwdkin
import math

In [2]:
ex = np.array([1,0,0])
ey = np.array([0,1,0])
ez = np.array([0,0,1])
l0 = 0.061 # base to servo 1
l1 = 0.0435 # servo 1 to servo 2
l2 = 0.08285 # servo 2 to servo 3
l3 = 0.08285 # servo 3 to servo 4
l4 = 0.07385 # servo 4 to servo 5
l5 = 0.05457 # servo 5 to gripper
P01 = ( l0 + l1 ) * ez 
P12 = np.zeros (3) # translation between 1 and 2 frame in 1 frame
P23 = l2 * ex # translation between 2 and 3 frame in 2 frame
P34 = - l3 * ez # translation between 3 and 4 frame in 3 frame
P45 = np.zeros (3) # translation between 4 and 5 frame in 4 frame
P5T = -( l4 + l5 ) * ex 

P = np.array([P01, P12, P23, P34, P45, P5T]).T
H = np.array([ez, -ey, -ey, -ey, -ex]).T
print(P,H)
limits = None 
dofbot = UR5Arm(P, H, limits)

[[ 0.       0.       0.08285 -0.       0.      -0.12842]
 [ 0.       0.       0.      -0.       0.      -0.     ]
 [ 0.1045   0.       0.      -0.08285  0.      -0.     ]] [[ 0  0  0  0 -1]
 [ 0 -1 -1 -1  0]
 [ 1  0  0  0  0]]


In [3]:
start = np.array([90, 90, 90, 90, 90])[None].T*math.pi/180
hover = [90, 40, 40, 90, 90]
hover_right = [65, 40, 40, 90, 90]
hover_2 = [62, 61, -13, 100, 90]

red = np.array([115, 31, 31, 90, 90])[None].T*math.pi/180
yellow = np.array([62, 31, 31, 90, 90])[None].T*math.pi/180
green = np.array([62, 31, 31, 90, 90])[None].T*math.pi/180

H_red = fwdkin(dofbot, red)
print(H_red)
block1 = np.array([90, 31, 31, 90, 90])[None].T*math.pi/180
block2 = np.array([90, 50, 0, 100, 90])[None].T*math.pi/180
block3 = np.array([90, 50, 0, 100, 90])[None].T*math.pi/180
print(fwdkin(dofbot, block3))


[[ 3.73149777e-01 -1.98407256e-01 -9.06307787e-01 -1.08848143e-01]
 [-8.00222279e-01  4.25485733e-01 -4.22618262e-01  2.33425596e-01]
 [ 4.69471563e-01  8.82947593e-01 -9.80268747e-17  4.79856474e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[-9.61481343e-17 -5.55111512e-17 -1.00000000e+00  2.53060592e-17]
 [-8.66025404e-01  5.00000000e-01  5.55111512e-17  2.27936718e-01]
 [ 5.00000000e-01  8.66025404e-01 -9.61481343e-17  5.05018286e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [4]:
def get_RP(H):
    return H[:3,:3], H[:3, -1]

In [5]:
from path_planning import path_planner
from Arm_Lib import Arm_Device

Arm = Arm_Device()

H_start = fwdkin(dofbot, start)
R_start, P_start = get_RP(H_start)
print(R_start, P_start)

H_red = fwdkin(dofbot, red)
R_red, P_red = get_RP(H_red)
print(R_red, P_red)

path_planner(Arm, dofbot, 5, R_start, P_start, R_red, P_red, 0.000001, 0.00000001, 0.00000001,90, 1500)
# path_planner(Arm, dofbot, 5, red, hover, 0.000001, 0.00000001, 0.00000001,135, 1500)
# path_planner(Arm, dofbot, 5, hover, block1, 0.000001, 0.00000001, 0.00000001,135, 1500)
# path_planner(Arm, dofbot, 5, block1, hover, 0.000001, 0.00000001, 0.00000001, 90, 1500)
# path_planner(Arm, dofbot, 5, hover_right, yellow, 0.000001, 0.00000001, 0.00000001, 90, 1500)
# path_planner(Arm, dofbot, 5, yellow, hover_2, 0.000001, 0.00000001, 0.00000001, 135, 1500)
# path_planner(Arm, dofbot, 5, hover_2, block2, 0.000001, 0.00000001, 0.00000001, 135, 1500)
# path_planner(Arm, dofbot, 5, hover, block2, 0.000001, 0.00000001, 0.00000001, 90, 1500)

#path_planner(5, red_open, start, 0.000001, 0.00000001, 0.00000001)
#path_planner(5, start, block1_closed, 0.000001, 0.00000001, 0.00000001)
#path_planner(5, block1_closed, start, 0.000001, 0.00000001, 0.00000001)
#path_planner(5, start, yellow_closed, 0.000001, 0.00000001, 0.00000001)
#Arm.Arm_serial_servo_write6(red[0], red[1], red[2], red[3], red[4], 135, 1500)
print("passed Path Following")

[[-3.69778549e-32 -2.22044605e-16 -1.00000000e+00]
 [-3.33066907e-16 -1.00000000e+00  2.22044605e-16]
 [-1.00000000e+00  3.33066907e-16 -3.69778549e-32]] [7.81231141e-33 7.03670455e-17 3.98620000e-01]
[[ 3.73149777e-01 -1.98407256e-01 -9.06307787e-01]
 [-8.00222279e-01  4.25485733e-01 -4.22618262e-01]
 [ 4.69471563e-01  8.82947593e-01 -9.80268747e-17]] [-0.10884814  0.2334256   0.04798565]
[[-0.5566704  -0.24620194 -0.79341204]
 [-0.66341395  0.70658796  0.24620194]
 [ 0.5         0.66341395 -0.5566704 ]] [0.15816513 0.18849386 0.11814353]
[-5.93641222e-08  1.31662043e-08 -3.82107213e-09  2.14878152e-05
  9.68848149e-05 -7.24935068e-07]
Inverse Kinematics Converged after 166 iterations
[1.35254194 1.57309424 1.56500276 1.57429191 1.78905071]
q_0 converged
[[-0.5566704  -0.24620194 -0.79341204]
 [-0.66341395  0.70658796  0.24620194]
 [ 0.5         0.66341395 -0.5566704 ]] [0.15816513 0.18849386 0.11814353]
[-2.31169100e-05 -3.32140387e-05 -4.08460158e-05  4.41944037e-05
 -6.33025492e-05

# 